In [1]:
from obspy import read, Stream
import seisbench
import seisbench.models as sbm
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import pandas as pd

Se Carga el modelo pre-entrenado de eqtrnasformes. Hay otras opciones de versiones de este modelo entrenado con otros datos, pero algunas ya no estan disponibles.

In [2]:
seisbench.use_backup_repository()
eq_model = sbm.EQTransformer.from_pretrained("stead")

2025-02-06 18:04:37,611 | seisbench | WARNING | Setting remote root to: https://seisbench.gfz-potsdam.de/mirror/
Please note that this can affect your download speed.


Se seleciona el directorio donde se encuentran los archivos .mseed, estos tienen la caracteristica de que cada componente tiene su propio archivo.

In [ ]:
directory = "/Users/roberto/SDS2/RAE96/"

df_acumulado = pd.DataFrame()

Se selecciona el tango de fechas que se quieren procesar recordando que la nomenclatura se encuentra en dias del año.

In [ ]:
for i in range(122,289):
    print(i)
    pattern_E = directory + "RAE96_EHE_2024_"+str(i)+".mseed"
    pattern_N = directory + "RAE96_EHN_2024_"+str(i)+".mseed"
    pattern_Z = directory + "RAE96_EHZ_2024_"+str(i)+".mseed"

    # Para automarizar el Proceso, hay veces que los dias no se encuentran en la carpeta, y de esta manera se evita que los archivos faltantes paren la secuencia {
    try:
        stream_E = read(pattern_E)
    except FileNotFoundError:
        print(f"File {pattern_E} not found")
        stream_E = None

    try:
        stream_N = read(pattern_N)
    except FileNotFoundError:
        print(f"File {pattern_N} not found")
        stream_N = None

    try:
        stream_Z = read(pattern_Z)
    except FileNotFoundError:
        print(f"File {pattern_Z} not found")
        stream_Z = None

    ##### }
    # Ya que necestiamos todas las trazas para poder hacer este análisis se ponen estas condiciones para construir un objeto stream.{
    #(cabe aclarar que en caso de que solo se cuente con una traza Z, esta se puede duplicar para que el stream siga conteniendo 3 trazas)

    if stream_E is not None:
        if stream_N is not None:
            if stream_Z is not None:
                stream = stream_E + stream_N + stream_Z
    ##}
    # Muchas veces la traza tiene el problema de estar segmentada, y asumiendo que solo se procesara cuando se tengan las 3 componentes,
    # por la manera en que se organizan estas, se hace esto para seleccionar los segmentos de cada traza correspondiente y se hace otro stream {
    every = int(len(stream)/3)
    for i in range(0,every):
        selected_stream = Stream()
        # Add traces by specifying their index or other identifying feature
        # For example, if you know the indices:
        selected_stream += stream[i]  #  trace of component E
        selected_stream += stream[i+every]  #  trace of component N
        selected_stream += stream[i+2*every]  #  trace of component Z
    ##}

    # Ahora que se creo el stream con el que se va a trabajar es NECESARIO que todos estos conincidan en datos, 
    # por lo que se cortan los streams para que todos tengan la misma longitud.
    # De manera que El Maximo tiempo de la traza mas atrasada sea el inicio del stream
    # Y el minimo tiempo de la traza mas precoz sea el final.
        # Obtener el tiempo de inicio y fin común más corto entre las trazas {
        tiempo_inicio_comun = max(traza.stats.starttime for traza in selected_stream)
        tiempo_fin_comun = min(traza.stats.endtime for traza in selected_stream)
        if tiempo_inicio_comun <= tiempo_fin_comun:
            # Recortar el stream al tiempo común
            stream_trimmed = selected_stream.slice(starttime=tiempo_inicio_comun, endtime=tiempo_fin_comun)
    ##}
    # Se hace un Detrend al stream{}
            stream_trimmed.detrend(type="linear")
            #stream_trimmed.normalize()
    
    # Es en esta linea que se utiliza el modelo pre-entrenado de EQtransformers.{}
            outputs = eq_model.classify(stream_trimmed)
            print(outputs)
    # Lo que sigue es hace un objeto de dataframe para acomodar los resultados y poder ir acumulando los resultados en otro dataframe.
            ############Convirtiendo a dataframes
            df = pd.DataFrame(outputs.picks)
    # Se verifica que EQtransformers en verdad dio un resultado 
            if not df.empty:
                df = df[0].apply(lambda x: x.__dict__)
                df = pd.DataFrame(df.tolist())
                df.rename(columns={"start_time":'start_time_picks'},inplace=True)
                df.rename(columns={"end_time":'end_time_picks'},inplace=True)
                df_P = df[df['phase']=='P']
                df_S = df[df['phase']=='S']
    # Lo anterior fue la parte de los picados, pero tambien nos importa el rango de detenccion, que es lo que sigue a continuacion{}
                df2 = pd.DataFrame(outputs.detections)
                df2 = df2[0].apply(lambda x: x.__dict__)
                df2 = pd.DataFrame(df2.tolist())
                df2.rename(columns={"start_time":'start_time_detections'},inplace=True)
                df2.rename(columns={"end_time":'end_time_detections'},inplace=True)
                df2.rename(columns={"peak_value":'peak_value_detections'},inplace=True)
    
    # Se hace un merge complicado de los dataframes para que coincidan los picados de P y S con un rango de deteccion correspondiente
    # Esto se hizo de manera artesanal viendo ejemplos de las detencciones.
                ############## Merge de los dataframes
                final_merged_df = df2.merge(
                    df_P,  # Phase P
                    on="trace_id",
                    how="inner"
                ).merge(
                    df_S,  # Phase S
                    on="trace_id",
                    how="inner",
                    suffixes=("_P", "_S")
                ).query("start_time_detections <= end_time_picks_P <= end_time_detections and "
                        "start_time_detections <= end_time_picks_S <= end_time_detections")
                final_merged_df.reset_index(drop=True, inplace=True)

                final_merged_df['archivo'] = "RAE96_EHE_2024_"+str(i)+".mseed"
                #final_merged_df
                ##############################
                df_acumulado = pd.concat([df_acumulado, final_merged_df], ignore_index=True)


        else:
            # Si no hay solapamiento, puedes manejar el caso aquí (por ejemplo, imprimir un mensaje o continuar)
            print(f"Rango de tiempo inválido: {tiempo_inicio_comun}, {tiempo_fin_comun} ")

df_acumulado.to_csv("Picados_loscabos_eqtransformes.csv",index=False)      
